In [3]:
%%capture
!pip install wandb -qqq

In [2]:
import tensorflow.keras as keras
from google.colab import drive
import numpy as np
import cv2
import wandb
from wandb.keras import WandbCallback

MODELS_PATH = "/content/gdrive/My Drive/magshimim/camerona/models/"
BASE = "beard_base_model.h5"
BEST = "beard_best_model.h5"
DATASET_PATH_DRIVE = "/content/gdrive/My Drive/magshimim/camerona/dataset/beard/"
DATASET_PATH_COLAB = "/content/beard/"
VAL_TEXT = DATASET_PATH_COLAB+"val.txt"
TRAIN_TEXT = DATASET_PATH_COLAB+"train.txt"
TEST_TEXT = DATASET_PATH_COLAB+"test.txt"

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
!ln -s "/content/gdrive/My Drive/magshimim/camerona/dataset/beard/" "/content/"

In [4]:
 !wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [1]:
def buildModel(lr = 0.001):
  input_layer = keras.layers.Input(shape=[32,32,3])

  #block1
  layer = keras.layers.Conv2D(32, (3,3), strides=(1,1), activation='relu', padding='same', name='block1_conv1')(input_layer)
  layer = keras.layers.MaxPool2D(2, name='block1_max1')(layer)

  #block2
  layer = keras.layers.Conv2D(16, (3,3), strides=(1,1), activation='relu', padding='same', name='block2_conv1')(layer)
  layer = keras.layers.MaxPool2D(2, name='block2_max1')(layer)

  #output block
  layer = keras.layers.Flatten()(layer)
  layer = keras.layers.Dense(16, activation="relu")(layer)
  layer = keras.layers.Dense(1, activation="sigmoid", name='output_layer')(layer)
 
  model = keras.Model(inputs=input_layer, outputs=layer)
  opt = keras.optimizers.Adam(learning_rate=lr)
  model.compile(optimizer = opt, loss = ['binary_crossentropy'], metrics=[keras.metrics.BinaryAccuracy()])
  print(model.summary())

  return model

In [7]:
def getData(file_path):
  f = open(file_path, 'r')
  data = f.read().split('\n')
  f.close()
  x = []
  y = []
  all = len(data)
  cnt = 1
  for line in data:
    if cnt%250 == 0:
      print("{0}/{1}".format(cnt, all))
    cnt += 1
    line_data = line.split(" ")
    line_data[0] = line_data[0].replace('\\', '/')
    try:
      img = cv2.imread(DATASET_PATH_COLAB+line_data[0])
      img = cv2.resize(img, (64,64))/255
    except:
      continue
    x.append(img)
    y.append(int(line_data[1]))
  x = np.array(x)
  y = np.array(y)
  return x,y

In [8]:
#building and saving base model
model = buildModel()
model.save(MODELS_PATH+BASE)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 32)        896       
_________________________________________________________________
block1_max1 (MaxPooling2D)   (None, 32, 32, 32)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 16)        4624      
_________________________________________________________________
block2_max1 (MaxPooling2D)   (None, 16, 16, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0         
_________________________________________________________________
dense (Dense)                (None, 16)                65552 

In [9]:
wandb.init(project="beard-classification")

wandb: Currently logged in as: ofekh (use `wandb login --relogin` to force relogin)


In [10]:
#callbacks
best_callback = keras.callbacks.ModelCheckpoint(MODELS_PATH+BEST, save_best_only=True)
wandb_callback = WandbCallback()

In [11]:
#loading data
print("loading validation data")
val_x, val_y = getData(VAL_TEXT)
print("loading train data")
train_x, train_y = getData(TRAIN_TEXT)

loading validation data
250/768
500/768
750/768
loading train data
250/12285
500/12285
750/12285
1000/12285
1250/12285
1500/12285
1750/12285
2000/12285
2250/12285
2500/12285
2750/12285
3000/12285
3250/12285
3500/12285
3750/12285
4000/12285
4250/12285
4500/12285
4750/12285
5000/12285
5500/12285
5750/12285
6000/12285
6250/12285
6500/12285
6750/12285
7000/12285
7250/12285
7500/12285
7750/12285
8000/12285
8250/12285
8500/12285
8750/12285
9000/12285
9250/12285
9500/12285
9750/12285
10000/12285
10250/12285
10500/12285
10750/12285
11000/12285
11250/12285
11500/12285
11750/12285
12000/12285
12250/12285


In [2]:
model = buildModel(0.001)

NameError: ignored

In [13]:
model.fit(train_x,
          train_y,
          epochs=24, 
          validation_data=(val_x, val_y),
          callbacks=[best_callback, wandb_callback])

Epoch 1/24
384/384 [==============================] - 52s 132ms/step - loss: 0.4408 - binary_accuracy: 0.7866 - val_loss: 0.2165 - val_binary_accuracy: 0.9141
Epoch 2/24
384/384 [==============================] - 51s 133ms/step - loss: 0.2280 - binary_accuracy: 0.9012 - val_loss: 0.1709 - val_binary_accuracy: 0.9349
Epoch 3/24
384/384 [==============================] - 54s 141ms/step - loss: 0.1728 - binary_accuracy: 0.9314 - val_loss: 0.1752 - val_binary_accuracy: 0.9297
Epoch 4/24
384/384 [==============================] - 50s 131ms/step - loss: 0.1376 - binary_accuracy: 0.9453 - val_loss: 0.1269 - val_binary_accuracy: 0.9492
Epoch 5/24
384/384 [==============================] - 52s 136ms/step - loss: 0.1050 - binary_accuracy: 0.9568 - val_loss: 0.1464 - val_binary_accuracy: 0.9440
Epoch 6/24
384/384 [==============================] - 52s 136ms/step - loss: 0.0927 - binary_accuracy: 0.9640 - val_loss: 0.1070 - val_binary_accuracy: 0.9518
Epoch 7/24
384/384 [==========================

In [14]:
test_x, test_y = getData(TEST_TEXT)

250/2303
500/2303
750/2303
1000/2303
1250/2303
1500/2303
1750/2303
2000/2303
2250/2303


In [15]:
loss, acc = model.evaluate(x = test_x, y = test_y)

72/72 [==============================] - 3s 37ms/step - loss: 0.0580 - binary_accuracy: 0.9844


In [16]:
model.save(MODELS_PATH+BEST)